In [1]:
import os
import csv
import time
import pickle
import numpy as np
import pandas as pd
from fABBA import fABBA
from software.qabba import fastQABBA
from tqdm import tqdm
import matplotlib.pyplot as plt
from source.figdefault import figdefault
from tslearn.metrics import dtw as dtw
from fABBA import inverse_compress
from fABBA import inverse_digitize
from source.jabba.qabba import compute_storage
from sklearn.metrics import mean_squared_error as mse
import warnings
warnings.filterwarnings("ignore")

In [2]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class TimeSeriesTooShort(Error):
    """Raised when time series is too short"""
    pass

class NotEnoughPieces(Error):
    """Raised when not enough pieces"""
    pass

class CompressionTolHigh(Error):
    """Raised when not enough pieces"""
    pass

class VariantsError(Error):
    """Raised when not enough unique pieces"""
    # Making assumption Ckmeans is available and suitable tol and scl is given.
    pass

class UnknownError(Error):
    """Raised when not enough unique pieces"""
    # Making assumption Ckmeans is available and suitable tol and scl is given.
    pass


class QABBA1Error(Error):
    # Error arising from QABBA
    pass

class QABBA2Error(Error):
    # Error arising from QABBA
    pass

class QABBA3Error(Error):
    # Error arising from QABBA
    pass

class QABBA4Error(Error):
    # Error arising from QABBA
    pass

class QABBA5Error(Error):
    # Error arising from QABBA
    pass

class QABBA6Error(Error):
    # Error arising from QABBA
    pass

class QABBA7Error(Error):
    # Error arising from QABBA
    pass

class QABBA8Error(Error):
    # Error arising from QABBA
    pass

class fABBAError(Error):
    # Error arising from fABBA
    pass



datadir = 'UCR2018/'
# tolerances
tol = [0.05*i for i in range(1,11)]

# If pickle file does not exist, make it.
# if not os.path.exists('scl'+str(scl)+'.pkl'):

# Calculate number of time series, to provide progess information.
ts_count = 0
for root, dirs, files in os.walk(datadir):
    for file in files:
        if file.endswith('tsv'):
            with open(os.path.join(root, file)) as f:
                content = f.readlines()
                ts_count += len(content)
                
print('Number of time series:', ts_count)


# Construct list of NaNs
D_fABBA_2 = ts_count*[np.NaN]
D_QABBA1_2 = ts_count*[np.NaN]
D_QABBA1_2 = ts_count*[np.NaN]
D_QABBA2_2 = ts_count*[np.NaN]
D_QABBA3_2 = ts_count*[np.NaN]
D_QABBA4_2 = ts_count*[np.NaN]
D_QABBA5_2 = ts_count*[np.NaN]
D_QABBA6_2 = ts_count*[np.NaN]
D_QABBA7_2 = ts_count*[np.NaN]
D_QABBA8_2 = ts_count*[np.NaN]

D_fABBA_DTW = ts_count*[np.NaN]
D_QABBA1_DTW = ts_count*[np.NaN]
D_QABBA2_DTW = ts_count*[np.NaN]
D_QABBA3_DTW = ts_count*[np.NaN]
D_QABBA4_DTW = ts_count*[np.NaN]
D_QABBA5_DTW = ts_count*[np.NaN]
D_QABBA6_DTW = ts_count*[np.NaN]
D_QABBA7_DTW = ts_count*[np.NaN]
D_QABBA8_DTW = ts_count*[np.NaN]

D_fABBA_2_diff = ts_count*[np.NaN]
D_QABBA1_2_diff = ts_count*[np.NaN]
D_QABBA2_2_diff = ts_count*[np.NaN]
D_QABBA3_2_diff = ts_count*[np.NaN]
D_QABBA4_2_diff = ts_count*[np.NaN]
D_QABBA5_2_diff = ts_count*[np.NaN]
D_QABBA6_2_diff = ts_count*[np.NaN]
D_QABBA7_2_diff = ts_count*[np.NaN]
D_QABBA8_2_diff = ts_count*[np.NaN]

D_fABBA_DTW_diff = ts_count*[np.NaN]
D_QABBA1_DTW_diff = ts_count*[np.NaN]
D_QABBA2_DTW_diff = ts_count*[np.NaN]
D_QABBA3_DTW_diff = ts_count*[np.NaN]
D_QABBA4_DTW_diff = ts_count*[np.NaN]
D_QABBA5_DTW_diff = ts_count*[np.NaN]
D_QABBA6_DTW_diff = ts_count*[np.NaN]
D_QABBA7_DTW_diff = ts_count*[np.NaN]
D_QABBA8_DTW_diff = ts_count*[np.NaN]

ts_name = ts_count*[''] # time series name for debugging
count = ts_count*[0]
compression = ts_count*[np.NaN] # Store amount of compression
tol_used = ts_count*[np.NaN] # Store tol used
error = ts_count*[0] # track errors

csymbolicNum = list()
cpiecesNum = list()
ctsname = list()
ctolist = list()

Number of time series: 201761


In [3]:
# Run through time series
index = 0
BIT_LEN = 32

for root, dirs, files in os.walk(datadir):
    for file in files:
        if file.endswith('tsv'):
            print('file:', file)

            # bool to keep track of plots, one plot per classification.
            need_to_plot = True

            with open(os.path.join(root, file)) as tsvfile:
                tsvfile = csv.reader(tsvfile, delimiter='\t')

                for ind, column in tqdm(enumerate(tsvfile)):
                    ts_name[index] += str(file) + '_' + str(ind) # Save filename + index

                    ts = [float(i) for i in column] # convert to list of floats
                    ts = np.array(ts[1:]) # remove class information

                    # remove NaN from time series
                    ts = ts[~np.isnan(ts)]

                    # Z Normalise the times series
                    norm_ts = (ts -  np.mean(ts))
                    std = np.std(norm_ts, ddof=1)
                    std = std if std > np.finfo(float).eps else 1
                    norm_ts /= std
                    
                    try:
                        # Check length of time timeseries
                        if len(norm_ts) < 100:
                            raise(TimeSeriesTooShort)

                        # Reset tolerance
                        tol_index = 0

                        # ABBA (Adjust tolerance so at least 20% compression is used)
                        for tol_index in range(len(tol)):
                            abba = fABBA(tol=tol[tol_index],verbose=0)
                            pieces = abba.compress(norm_ts)
                            ABBA_len = len(pieces)
                            if ABBA_len <= len(norm_ts)/5:
                                tol_used[index] = tol[tol_index]
                                break
                            elif tol_index == len(tol)-1:
                                raise(CompressionTolHigh)
                                
                        pieces = np.array(pieces)
                        try:
                            fabba = fABBA(verbose=0, alpha=0.1, scl=1, sorting='2-norm') 
                            st = time.time()
                            symbols_fabba, parameters_fabba = fabba.digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_fabba = ed - st
                        except:
                            raise(fABBAError)
                            
                        symbolnum = len(set(symbols_fabba))
                        try:
                            qabba1 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0,
                                               bits_for_len=BIT_LEN, bits_for_inc=4)
                            st = time.time()
                            symbols_qabba1 = qabba1._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba1 = ed - st
                        except:
                            raise(QABBA1Error)
                         
                        try:
                            qabba2 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=8)
                            st = time.time()
                            symbols_qabba2 = qabba2._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba2 = ed - st
                        except:
                            raise(QABBA2Error)
                        
                        try:
                            qabba3 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0,
                                               bits_for_len=BIT_LEN, bits_for_inc=12)
                            st = time.time()
                            symbols_qabba3 = qabba3._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba3 = ed - st
                        except:
                            raise(QABBA3Error)
                            
                        
                        try:
                            qabba4 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=16)
                            st = time.time()
                            symbols_qabba4 = qabba4._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba4 = ed - st
                        except:
                            raise(QABBA4Error)

                        try:
                            qabba5 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=20)
                            
                            st = time.time()
                            symbols_qabba5 = qabba5._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba5 = ed - st
                        except:
                            raise(QABBA5Error)
                            
                        try:
                            qabba6 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=24)
                            
                            st = time.time()
                            symbols_qabba6 = qabba6._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba6 = ed - st
                        except:
                            raise(QABBA6Error)
                            
                            
                        try:
                            qabba7 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=28)
                            
                            st = time.time()
                            symbols_qabba7 = qabba7._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba7 = ed - st
                        except:
                            raise(QABBA7Error)
                            
                        try:
                            qabba8 = fastQABBA(init='agg', sorting='2-norm', alpha=0.1, verbose=0, 
                                               bits_for_len=BIT_LEN, bits_for_inc=32)
                            
                            st = time.time()
                            symbols_qabba8 = qabba8._digitize(pieces[:,:2].astype(np.float32))
                            ed = time.time()
                            time_qabba8 = ed - st
                        except:
                            raise(QABBA8Error)
                            
                        csymbolicNum.append(symbolnum)
                        cpiecesNum.append(len(pieces))
                        ctsname.append(str(file) + '_' + str(ind))
                        ctolist.append(tol[tol_index])
                        
                        ts_fABBA = inverse_compress(inverse_digitize(symbols_fabba, parameters_fabba), norm_ts[0])
                        ts_QABBA1 = qabba1.inverse_transform(symbols_qabba1, norm_ts[0])
                        ts_QABBA2 = qabba2.inverse_transform(symbols_qabba2, norm_ts[0])
                        ts_QABBA3 = qabba3.inverse_transform(symbols_qabba3, norm_ts[0])
                        ts_QABBA4 = qabba4.inverse_transform(symbols_qabba4, norm_ts[0])
                        ts_QABBA5 = qabba5.inverse_transform(symbols_qabba5, norm_ts[0])
                        ts_QABBA6 = qabba6.inverse_transform(symbols_qabba6, norm_ts[0])
                        ts_QABBA7 = qabba7.inverse_transform(symbols_qabba7, norm_ts[0])
                        ts_QABBA8 = qabba8.inverse_transform(symbols_qabba8, norm_ts[0])
                        
                        # Compute distances
                        minlen1 = min(len(ts_QABBA1), len(norm_ts))
                        minlen2 = min(len(ts_QABBA2), len(norm_ts))
                        minlen3 = min(len(ts_QABBA3), len(norm_ts))
                        minlen4 = min(len(ts_QABBA4), len(norm_ts))
                        minlen5 = min(len(ts_QABBA5), len(norm_ts))
                        minlen6 = min(len(ts_QABBA6), len(norm_ts))
                        minlen7 = min(len(ts_QABBA7), len(norm_ts))
                        minlen8 = min(len(ts_QABBA8), len(norm_ts))
                        minlen9 = min(len(ts_fABBA), len(norm_ts))
                        
                        D_QABBA1_2[index] = mse(norm_ts[:minlen1],ts_QABBA1[:minlen1])
                        D_QABBA2_2[index] = mse(norm_ts[:minlen2],ts_QABBA2[:minlen2])
                        D_QABBA3_2[index] = mse(norm_ts[:minlen3],ts_QABBA3[:minlen3])
                        D_QABBA4_2[index] = mse(norm_ts[:minlen4],ts_QABBA4[:minlen4])
                        D_QABBA5_2[index] = mse(norm_ts[:minlen5],ts_QABBA5[:minlen5])
                        D_QABBA6_2[index] = mse(norm_ts[:minlen6],ts_QABBA6[:minlen6])
                        D_QABBA7_2[index] = mse(norm_ts[:minlen7],ts_QABBA7[:minlen7])
                        D_QABBA8_2[index] = mse(norm_ts[:minlen8],ts_QABBA8[:minlen8])
                        D_fABBA_2[index]  = mse(norm_ts[:minlen9],ts_fABBA[:minlen9])
                        
                        D_QABBA1_DTW[index]    = dtw(norm_ts[:minlen1], ts_QABBA1[:minlen1])
                        D_QABBA2_DTW[index]    = dtw(norm_ts[:minlen2], ts_QABBA2[:minlen2])
                        D_QABBA3_DTW[index]    = dtw(norm_ts[:minlen3], ts_QABBA3[:minlen3])
                        D_QABBA4_DTW[index]    = dtw(norm_ts[:minlen4], ts_QABBA4[:minlen4])
                        D_QABBA5_DTW[index]    = dtw(norm_ts[:minlen5], ts_QABBA5[:minlen5])
                        D_QABBA6_DTW[index]    = dtw(norm_ts[:minlen6], ts_QABBA6[:minlen6])
                        D_QABBA7_DTW[index]    = dtw(norm_ts[:minlen7], ts_QABBA7[:minlen7])
                        D_QABBA8_DTW[index]    = dtw(norm_ts[:minlen8], ts_QABBA8[:minlen8])
                        D_fABBA_DTW[index]     = dtw(norm_ts[:minlen9], ts_fABBA[:minlen9])
                        
                        D_QABBA1_2_diff[index]  = mse(np.diff(norm_ts[:minlen1]),np.diff(ts_QABBA1[:minlen1]))
                        D_QABBA2_2_diff[index]  = mse(np.diff(norm_ts[:minlen2]),np.diff(ts_QABBA2[:minlen2]))
                        D_QABBA3_2_diff[index]  = mse(np.diff(norm_ts[:minlen3]),np.diff(ts_QABBA3[:minlen3]))
                        D_QABBA4_2_diff[index]  = mse(np.diff(norm_ts[:minlen4]),np.diff(ts_QABBA4[:minlen4]))
                        D_QABBA5_2_diff[index]  = mse(np.diff(norm_ts[:minlen5]),np.diff(ts_QABBA5[:minlen5]))
                        D_QABBA6_2_diff[index]  = mse(np.diff(norm_ts[:minlen6]),np.diff(ts_QABBA6[:minlen6]))
                        D_QABBA7_2_diff[index]  = mse(np.diff(norm_ts[:minlen7]),np.diff(ts_QABBA7[:minlen7]))
                        D_QABBA8_2_diff[index]  = mse(np.diff(norm_ts[:minlen8]),np.diff(ts_QABBA8[:minlen8]))
                        D_fABBA_2_diff[index]   = mse(np.diff(norm_ts[:minlen9]),np.diff(ts_fABBA[:minlen9]))
                        
                        D_QABBA1_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen1]), np.diff(ts_QABBA1[:minlen1]))
                        D_QABBA2_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen2]), np.diff(ts_QABBA2[:minlen2]))
                        D_QABBA3_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen3]), np.diff(ts_QABBA3[:minlen3]))
                        D_QABBA4_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen4]), np.diff(ts_QABBA4[:minlen4]))
                        D_QABBA5_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen5]), np.diff(ts_QABBA5[:minlen5]))
                        D_QABBA6_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen6]), np.diff(ts_QABBA6[:minlen6]))
                        D_QABBA7_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen7]), np.diff(ts_QABBA7[:minlen7]))
                        D_QABBA8_DTW_diff[index]   =  dtw(np.diff(norm_ts[:minlen8]), np.diff(ts_QABBA8[:minlen8]))                                         
                        D_fABBA_DTW_diff[index]    =  dtw(np.diff(norm_ts[:minlen9]), np.diff(ts_fABBA[:minlen9]))
                        
                        compression[index] = ABBA_len/len(norm_ts) # Store compression amount
                        count[index] = len(norm_ts)
                        index += 1
                        
                    except(TypeError):
                        error[index] = 1
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                        
                    except(TimeSeriesTooShort):
                        error[index] = 2
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass

                    except(NotEnoughPieces):
                        error[index] = 3
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass

                    except(CompressionTolHigh):
                        error[index] = 4
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass

                    except(UnknownError):
                        error[index] = 5
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(ValueError):
                        error[index] = 6
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass

                    
                    except(fABBAError):
                        error[index] = 8
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA1Error):
                        error[index] = 9
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA2Error):
                        error[index] = 10
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA3Error):
                        error[index] = 11
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA4Error):
                        error[index] = 12
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    
                    except(QABBA5Error):
                        error[index] = 13
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA6Error):
                        error[index] = 14
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA7Error):
                        error[index] = 15
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
                    except(QABBA8Error):
                        error[index] = 16
                        compression[index] = np.NaN
                        tol_used[index] = np.NaN
                        index += 1
                        pass
                    
D = {}
D['ts_name'] = ts_name
D['compression'] = compression
D['tol_used'] = tol_used
D['error'] = error
D['tol'] = tol

D['QABBA1_MSE'] = D_QABBA1_2
D['QABBA2_MSE'] = D_QABBA2_2
D['QABBA3_MSE'] = D_QABBA3_2
D['QABBA4_MSE'] = D_QABBA4_2
D['QABBA5_MSE'] = D_QABBA5_2
D['QABBA6_MSE'] = D_QABBA6_2
D['QABBA7_MSE'] = D_QABBA7_2
D['QABBA8_MSE'] = D_QABBA8_2
D['fABBA_MSE'] = D_fABBA_2

D['QABBA1_DTW'] = D_QABBA1_DTW
D['QABBA2_DTW'] = D_QABBA2_DTW
D['QABBA3_DTW'] = D_QABBA3_DTW
D['QABBA4_DTW'] = D_QABBA4_DTW
D['QABBA5_DTW'] = D_QABBA5_DTW
D['QABBA6_DTW'] = D_QABBA6_DTW
D['QABBA7_DTW'] = D_QABBA7_DTW
D['QABBA8_DTW'] = D_QABBA8_DTW
D['fABBA_DTW'] = D_fABBA_DTW

D['QABBA1_MSE_diff'] = D_QABBA1_2_diff
D['QABBA2_MSE_diff'] = D_QABBA2_2_diff
D['QABBA3_MSE_diff'] = D_QABBA3_2_diff
D['QABBA4_MSE_diff'] = D_QABBA4_2_diff
D['QABBA5_MSE_diff'] = D_QABBA5_2_diff
D['QABBA6_MSE_diff'] = D_QABBA6_2_diff
D['QABBA7_MSE_diff'] = D_QABBA7_2_diff
D['QABBA8_MSE_diff'] = D_QABBA8_2_diff
D['fABBA_MSE_diff'] = D_fABBA_2_diff

D['QABBA1_DTW_diff'] = D_QABBA1_DTW_diff
D['QABBA2_DTW_diff'] = D_QABBA2_DTW_diff
D['QABBA3_DTW_diff'] = D_QABBA3_DTW_diff
D['QABBA4_DTW_diff'] = D_QABBA4_DTW_diff
D['QABBA5_DTW_diff'] = D_QABBA5_DTW_diff
D['QABBA6_DTW_diff'] = D_QABBA6_DTW_diff
D['QABBA7_DTW_diff'] = D_QABBA7_DTW_diff
D['QABBA8_DTW_diff'] = D_QABBA8_DTW_diff
D['fABBA_DTW_diff'] = D_fABBA_DTW_diff

with open('results/bits/bitsINC0.1'+'.p', 'wb') as handle:
    pickle.dump(D, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
Datastore = pd.DataFrame(columns=["ts name", "number of pieces", "number of symbols","tol"])
Datastore["ts name"] = ctsname
Datastore["number of pieces"] = cpiecesNum 
Datastore["number of symbols"] = csymbolicNum  
Datastore["tol"] = ctolist 
Datastore.to_csv("results/bits/QABBAINC0.1sl.csv",index=False)

file: DodgerLoopWeekend_TRAIN.tsv


20it [00:02,  9.48it/s]


file: DodgerLoopWeekend_TEST.tsv


138it [00:05, 23.66it/s]


file: Wine_TRAIN.tsv


...

file: GunPointOldVersusYoung_TRAIN.tsv


136it [00:02, 52.64it/s]


file: GunPointOldVersusYoung_TEST.tsv


315it [00:05, 52.54it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'results/bits/bitsINC0.1.p'